In [1]:
!pwd

/Users/sushmitha047/Documents/RealWorldML-Pau/taxi_demand_predictor/notebooks


In [2]:
import os
os.chdir("..")

In [31]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "final_data.parquet")
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,25.0,29.0,34.0,31.0,32.0,8.0,6.0,4.0,0.0,1.0,...,0.0,6.0,5.0,1.0,4.0,2.0,2.0,2024-01-29,4,0.0
1,1.0,2.0,1.0,0.0,0.0,1.0,0.0,3.0,6.0,3.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2024-01-30,4,1.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,4.0,5.0,...,1.0,0.0,1.0,4.0,1.0,4.0,6.0,2024-01-31,4,2.0
3,1.0,1.0,2.0,1.0,0.0,1.0,1.0,3.0,2.0,1.0,...,3.0,3.0,3.0,1.0,5.0,3.0,0.0,2024-02-01,4,5.0
4,2.0,4.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,...,2.0,2.0,2.0,4.0,4.0,12.0,7.0,2024-02-02,4,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-26,110,0.0
80737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-27,110,0.0
80738,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-28,110,0.0
80739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-29,110,0.0


In [54]:
from src.data_split import train_test_split
from datetime import datetime

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2024, 8, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

In [55]:
print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_test.shape=}")
print(f"{y_test.shape=}")

X_train.shape=(48655, 674)
y_train.shape=(48655,)
X_test.shape=(32086, 674)
y_test.shape=(32086,)


## Creating baseline model 1: predicting previous 1 hour rides

#### Inroder to predict taxi demand for the next hour, we'll look at what was the demand for previous hour

In [56]:
import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_1_hour']

In [57]:
model = BaselineModelPreviousHour() # created model object
predictions = model.predict(X_test)
predictions

0         4.0
1        13.0
2        13.0
3        20.0
4         3.0
         ... 
32081     0.0
32082     0.0
32083     0.0
32084     0.0
32085     0.0
Name: rides_previous_1_hour, Length: 32086, dtype: float32

In [58]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae=:.4f}")


test_mae=7.5164


## Creating baseline model 2: predicting previous 1 week rides

In [59]:
class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t - 7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f"rides_previous_{7*24}_hour"]

In [60]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)
predictions

0         1.0
1        12.0
2        39.0
3        42.0
4         3.0
         ... 
32081     0.0
32082     0.0
32083     0.0
32084     0.0
32085     0.0
Name: rides_previous_168_hour, Length: 32086, dtype: float32

In [61]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae=:.4f}")

test_mae=4.2525


## Creating baseline model 3: predicting average of last 4 weeks

In [62]:
class BaselineModelLast4Weeks:
    """
    Prediction = actual demand observed at t - 7 days, t - 14 days, t - 21 days, t - 28 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame):
        # 0.25 - for the average of 4
        return 0.25*(
            X_test[f"rides_previous_{7*24}_hour"] + \
            X_test[f"rides_previous_{2*7*24}_hour"] + \
            X_test[f"rides_previous_{3*7*24}_hour"] + \
            X_test[f"rides_previous_{4*7*24}_hour"]
        )

In [63]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)
predictions

0         4.00
1        12.25
2        33.50
3        38.25
4         2.00
         ...  
32081     0.00
32082     0.00
32083     0.00
32084     0.00
32085     0.00
Length: 32086, dtype: float32

In [64]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae=:.4f}")

test_mae=3.9177
